In [79]:
import numpy as np
import os
import codecs
import re
import string

# Load the GloVe File

In [80]:
! pip install util

ERROR: Could not find a version that satisfies the requirement util (from versions: none)
ERROR: No matching distribution found for util

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [81]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Get the set of stop words from the loaded spaCy model
stop_words = nlp.Defaults.stop_word
filename = 'glove.6B.300d.txt'
word_vector_dict = dict()

with open(filename,'r') as file:
    lines=file.readlines()
    for line in lines:
        input=line.split()
        if(input[0] in stop_words):
            continue
        else:
            word_vector_dict[input[0]]=[float(i) for i in input[1:]]
        break

In [82]:
word_vector_dict['the'][0:5]

[0.04656, 0.21318, -0.0074364, -0.45854, -0.035639]

# Load the articles

In [9]:
! wget https://s3-us-west-1.amazonaws.com/msan692/bbc.zip

--2024-03-23 15:23:06--  https://s3-us-west-1.amazonaws.com/msan692/bbc.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.113.120, 52.219.113.168, 52.219.120.120, ...
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.113.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874114 (2.7M) [application/zip]
Saving to: ‘bbc.zip.1’

bbc.zip.1           100%[===================>]   2.74M  --.-KB/s    in 0.1s    

2024-03-23 15:23:07 (20.8 MB/s) - ‘bbc.zip.1’ saved [2874114/2874114]



In [83]:
article_files = []
for path, subdirs, files in os.walk('bbc'):
    for name in files:
        article_files.append(os.path.join(path, name))

In [84]:
article_files[:5]

['bbc/COPYRIGHT',
 'bbc/entertainment/289.txt',
 'bbc/entertainment/262.txt',
 'bbc/entertainment/276.txt',
 'bbc/entertainment/060.txt']

In [85]:
# article_files=article_files[:100]
len(article_files)


2227

# Find centroid

In [86]:
def words(text):
    """
    Given a string, return a list of words normalized as follows.
    
        1. Lowercase all words
        2. Use re.sub function and string.punctuation + '0-9\\r\\t\\n\\n\n]'
            to replace all those char with a space character.
        3. Split on space to get word list.
        4. Ignore words < 3 char long.
        5. Remove English stop words
    Don't use Spacy.
    """
    text = text.lower()
    text = re.sub("[" + string.punctuation + '0-9\\r\\t\\n\\n\n]', ' ', text)
    text=re.sub(r'\n\n+', '\n', text)
    words = text.split(" ")
    output = [w for w in words if len(w) >= 3] 
    words_cleaned=[]
    for w in output:
        if ((w == '')):
            continue
        else:
            words_cleaned.append(w)
    return words_cleaned

In [87]:
article_info=list()
for f in article_files:
        if (f.endswith(".txt")):
            # print(f)
            filename=f
            filename = codecs.open(filename, encoding='latin-1', mode='r')
            text = filename.read()
            filename.close()
            # print(text[0:90])
            # print(1)
            str_output=''
            l=list()
            for s in text:  
                str_output=str_output+s
            text_set= words(str_output)
            for w in text_set:
                if w in word_vector_dict.keys():
                    l.append(word_vector_dict[w])
            centroid=np.mean(l, axis=0)
            record=(f,centroid)
            article_info.append(record)

In [88]:
article_info[0]

('bbc/entertainment/289.txt',
 array([ 4.6560e-02,  2.1318e-01, -7.4364e-03, -4.5854e-01, -3.5639e-02,
         2.3643e-01, -2.8836e-01,  2.1521e-01, -1.3486e-01, -1.6413e+00,
        -2.6091e-01,  3.2434e-02,  5.6621e-02, -4.3296e-02, -2.1672e-02,
         2.2476e-01, -7.5129e-02, -6.7018e-02, -1.4247e-01,  3.8825e-02,
        -1.8951e-01,  2.9977e-01,  3.9305e-01,  1.7887e-01, -1.7343e-01,
        -2.1178e-01,  2.3617e-01, -6.3681e-02, -4.2318e-01, -1.1661e-01,
         9.3754e-02,  1.7296e-01, -3.3073e-01,  4.9112e-01, -6.8995e-01,
        -9.2462e-02,  2.4742e-01, -1.7991e-01,  9.7908e-02,  8.3118e-02,
         1.5299e-01, -2.7276e-01, -3.8934e-02,  5.4453e-01,  5.3737e-01,
         2.9105e-01, -7.3514e-03,  4.7880e-02, -4.0760e-01, -2.6759e-02,
         1.7919e-01,  1.0977e-02, -1.0963e-01, -2.6395e-01,  7.3990e-02,
         2.6236e-01, -1.5080e-01,  3.4623e-01,  2.5758e-01,  1.1971e-01,
        -3.7135e-02, -7.1593e-02,  4.3898e-01, -4.0764e-02,  1.6425e-02,
        -4.4640e-01, 

# Recommend

In [89]:
def recommended(article, article_info, n):
    """ Return top n articles closest to article.

    Inputs:
        article: list [filename, title, text-minus-title, wordvec-centroid]
        articles: list of list [filename, title, text-minus-title, wordvec-centroid]

    Output:
         list of [topic, filename, title]
    """
    dist=[(np.linalg.norm(article[1] - a[1]), a) for a in article_info]
    sorted_dist=sorted(dist, key=lambda x: x[0])
    sorted_dist=sorted_dist[1:]
    output=list()
    for i in range(n):
        title=sorted_dist[i][1][0].split("/")[-1]
        topic=sorted_dist[i][1][0].split("/")[-2]
        output.append((topic,title))
    return output

In [90]:
recommendations=dict()
for article in article_info:
        op=recommended(article, article_info, 5)
        recommendations[(article[0].split("/")[-2], article[0].split("/")[-1])]=op

In [97]:
recommendations['entertainment', '275.txt']

[('politics', '263.txt'),
 ('politics', '040.txt'),
 ('politics', '327.txt'),
 ('tech', '301.txt'),
 ('politics', '224.txt')]